## Case 2: Asset Allocation at the Cook County Pension Fund

Download the file `case #2.csv` from the section `Modules/Week 8`. Make sure that the file is in the same data folder that you typically use for the other codes. 

The file contains monthly data from January 1990 to June 2023. The columns are:
1. `US_EQ`: Return on the US stock market.
2. `DMXUS_EQ`: Return on developed equity markets, excluding the US.
3. `10Y_TR`: Return on 10-year Treasury bonds.
4. `30Y_TR`: Return on 30-year Treasury bonds.
5. `HFRIFWI`: Return on the hedge fund index.
6. `TBILL`: Return on -month T-bills.

We start by initializing Python.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

We then load the data and print the first couple of lines to understand the structure of the data.
- Make sure to set the path to your data correctly as in the previous problem sets.

In [2]:
# Load data
df = pd.read_csv("../Data/case #2.csv", index_col="date")

print(df.tail())

          US_EQ  DMXUS_EQ   10Y_TR   30Y_TR  HFRIFWI   TBILL
date                                                        
202302 -0.02466  -0.02365 -0.03607 -0.07755  -0.0064  0.0034
202303  0.02395   0.03165  0.04802  0.08443  -0.0080  0.0036
202304  0.00898   0.01799  0.00689  0.00306   0.0024  0.0035
202305  0.00071  -0.00936 -0.01682 -0.04952  -0.0020  0.0036
202306  0.06618   0.06077 -0.01373  0.00321   0.0214  0.0040


**Question 1:** 
- Construct the return on the strategic benchmark for the Cook County Pension Fund. Call the newly constructed variable `ret_sb`. 
    - The most recent strategic asset allocation (as of June 22) invests 42% to US Equities, 27% to International Equities, and 31% to US Fixed Income, see https://www.cookcountypension.com/assets/1/6/3Q2022_Cook_County_Web_Doc.pdf. 
    - Hint: You will need to use this information to construct `ret_sb`

We use `US_EQ` for US Equities, `DMXUS_EQ` for International Equities, and `10Y_TR` for US Fixed Income. Report the summary statistics on the various investment opportunities and on the strategic benchmark. 

In [4]:
# Compute the return on the strategy benchmark 
## Insert you code here

# Compute the return on the strategic benchmark
df['ret_sb'] = 0.42 * df['US_EQ'] + 0.27 * df['DMXUS_EQ'] + 0.31 * df['10Y_TR']

# Compute summary statistics
summary = df[['US_EQ','DMXUS_EQ','10Y_TR','HFRIFWI','ret_sb']].describe().T[['mean', 'std']]

# Annualize the mean
summary['mean'] = summary['mean'] * 12

# Annualize the standard deviation
summary['std'] = summary['std'] * np.sqrt(12)

# Print the mean, standard deviation, and Sharpe ratio
print(summary.round(3))

           mean    std
US_EQ     0.104  0.153
DMXUS_EQ  0.081  0.152
10Y_TR    0.059  0.091
HFRIFWI   0.113  0.069
ret_sb    0.084  0.105


**Question 2:** The Cook County Pension Fund is considering an allocation to hedge funds. We analyze the performance of the hedge fund index relative to the strategic benchmark using performance regressions. <br>
1. Create two new variables, one for the excess return on the strategic benchmark and one for the excess return on the hedge fund index. To compute excess returns, use the 30-day T-bill return, `TBILL`.<br>
2. Compute the alpha of the hedge fund index relative to the strategic benchmark by regressing the excess return on the hedge fund index on the excess return on the strategic benchmark. As in Case #1, you can regress `y` on `x` and summarize the results using the following code:<br>
`reg_results = smf.ols(formula='y ~ x', data=df).fit()`<br>
`print(summary_col(reg_results, stars=True))`<br>
3. Discuss whether it is attractive to add hedge funds to the strategic allocation.

In [5]:
# Compute excess returns on the strategic benchmark and on the hedge fund index, relative to the 1-month T-bill rate
import statsmodels.formula.api as smf

# Calculate excess returns
df['excess_ret_sb'] = df['ret_sb'] - df['TBILL']
df['excess_ret_hf'] = df['HFRIFWI'] - df['TBILL']

# Regression analysis
reg_results = smf.ols(formula='excess_ret_hf ~ excess_ret_sb', data=df).fit()
print(reg_results.summary())

                            OLS Regression Results                            
Dep. Variable:          excess_ret_hf   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.578
Method:                 Least Squares   F-statistic:                     550.9
Date:                Tue, 14 Nov 2023   Prob (F-statistic):           3.16e-77
Time:                        11:03:51   Log-Likelihood:                 1187.8
No. Observations:                 402   AIC:                            -2372.
Df Residuals:                     400   BIC:                            -2364.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0049      0.001      7.752


**Question 3:** There has been a lot of discussion about the performance of hedge funds during the last decade. Re-run the performance regression as in the previous question but use the sample from January 2011 to June 2023. <br><br>
You can select the sample using the code:<br>
`df_short = df.loc['DATE':]`

where `'DATE'` is a number in the yyyymm format.

In [6]:
# Filter the DataFrame for the dates from January 2011 to June 2023
df_short = df.loc['201101':'202306']

# Calculate excess returns for the shortened DataFrame
df_short['excess_ret_sb'] = df_short['ret_sb'] - df_short['TBILL']
df_short['excess_ret_hf'] = df_short['HFRIFWI'] - df_short['TBILL']

# Perform the regression analysis on the shortened DataFrame
reg_results_short = smf.ols(formula='excess_ret_hf ~ excess_ret_sb', data=df_short).fit()
print(reg_results_short.summary())


                            OLS Regression Results                            
Dep. Variable:          excess_ret_hf   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     332.8
Date:                Tue, 14 Nov 2023   Prob (F-statistic):           1.07e-39
Time:                        11:10:40   Log-Likelihood:                 485.62
No. Observations:                 150   AIC:                            -967.2
Df Residuals:                     148   BIC:                            -961.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0002      0.001      0.195

/var/folders/54/fh5svs6574d0_v3wny9y75qh0000gn/T/ipykernel_25830/4214916554.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['excess_ret_sb'] = df_short['ret_sb'] - df_short['TBILL']
/var/folders/54/fh5svs6574d0_v3wny9y75qh0000gn/T/ipykernel_25830/4214916554.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_short['excess_ret_hf'] = df_short['HFRIFWI'] - df_short['TBILL']


In the absence of liabilities, the 1-month T-bill rate is typically used as the risk-free rate, for instance in evaluating mutual funds and ETFs. However, long-term investors with liabilities, like pension funds and insurance companies, care about the performance of their assets relative to the performance of their liabilities. For pension funds, the liabilities are typically paid far into the future, and the return on the value of the liabilities is therefore similar to the return on a long-term bond. We will use the return on a 30-year Treasury bond as the return on liabilities. 

**Question 4:** Compute two new variables: one for the excess return on the strategic benchmark and one for the excess return on the hedge fund index. To compute the excess return, subtract the return on the 30-year bond, `30Y_TR`. Then re-run the performance regression *for the full sample* from January 1990 to June 2023. Discuss the results. Should the Cook County Pension Fund invest capital to hedge funds?


In [7]:
# Assuming df is your full DataFrame from January 1990 to June 2023
# Calculate the excess returns
df['excess_ret_sb'] = df['ret_sb'] - df['30Y_TR']
df['excess_ret_hf'] = df['HFRIFWI'] - df['30Y_TR']

# Perform the regression analysis on the full sample
reg_results_full = smf.ols(formula='excess_ret_hf ~ excess_ret_sb', data=df).fit()
print(reg_results_full.summary())


                            OLS Regression Results                            
Dep. Variable:          excess_ret_hf   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     4998.
Date:                Tue, 14 Nov 2023   Prob (F-statistic):          3.87e-228
Time:                        11:21:25   Log-Likelihood:                 1001.0
No. Observations:                 402   AIC:                            -1998.
Df Residuals:                     400   BIC:                            -1990.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0025      0.001      2.457

For the following questions, you can use the material in the case, lecture notes, et cetera

**Question 5.** As many other defined-benefit pension plans, the Cook County Pension Fund is under-funded: The present value of their liabilities significantly exceeds the market value of assets. Discuss what the options are for the fund, and whether the fund should adjust its asset allocation?

**Question 6.** Macroeconomic conditions have changed significantly during the last three decades. A key development is the decline in interest rates since the eighties, as we have seen in class. Should the fund adjust its asset allocation?
 
**Question 7.** During the last year, another key recent development has been the global increase in inflation rates. Discuss whether you think that this could provide yet another challenge for the fund, or whether this could instead alleviate some of the pressures on the fund. 
